Denoising the lat-long detection data.

In [1]:
import numpy as np
import pandas as pd
from utils.map_display import *
from utils.data_ops import get_trajectory

In [2]:
# Import the post filter M2 data. We only need the set of id_tracks
track_path = "../data/tracks_tagged_v1.csv"
track_data = pd.read_csv(track_path)
track_data.head(5)
track_ids = track_data["id_track"].unique()

In [3]:
# Import our detection data frame. (Post Processed)
detection_path = "../data/detections_tagged_cached.csv"
detection_data = pd.read_csv(detection_path)

In [67]:
# Use a simple convolution for rolling average
def convolve(input, filter):
    # Do some checks on the filter
    if not np.allclose(np.sum(filter), 1):
        raise RuntimeError("Entires in filter must sum to 1")
    if len(filter) % 2 == 0:
        raise RuntimeError("Filter Length must be odd")
    # add padding to the input array
    pad_length = len(filter) // 2
    pad_head = np.repeat(input[0], pad_length)
    pad_tail = np.repeat(input[-1], pad_length)
    # add the padding
    padded_input = np.hstack((pad_head, input, pad_tail))
    # Compute the colvolution
    return np.convolve(padded_input, filter, mode = "valid")

def generate_gaussian_kernel(length, std : float = 1):
    assert length % 2 == 1
    
    def gaussian_density(x):
        c = 1/(np.sqrt(2 * np.pi) * std)
        return c * np.exp(- 0.5 * (x**2) / (std ** 2))
    
    mid = length // 2
    result = [gaussian_density(x) for x in range(-mid, mid + 1)]
    return result / np.sum(result) # Normalize to make sure things sum to 1.


def denoise_filter(input, filter_length : int = 3, std : float = 2**31):
    """
    denoise filter via gaussian convolution

    Args:
        input: input lat long array
        filter_length: length of the smoothing filter. Defaults to 3.
        std: standard deviation of the gaussian kernel. Defaults to a simple average
    """
    filter = generate_gaussian_kernel(filter_length, std)
    
    lat, long = input[:,0], input[:, 1]
    lat_smoothed, long_smoothed = convolve(lat, filter), convolve(long, filter)
    return np.vstack((lat_smoothed, long_smoothed)).T
    
    
    

In [21]:
# Get a random track id
id = np.random.choice(track_ids)
raw_detections = get_trajectory(detection_data, id)
location_center = np.mean(raw_detections, axis = 0)

In [64]:
m = init_map(location_center)
plot_trajectory(m, raw_detections)

In [72]:
print(generate_gaussian_kernel(15, 1))
m = init_map(location_center)
plot_trajectory(m, denoise_filter(raw_detections, 15, 10))

[9.13472036e-12 6.07588282e-09 1.48671951e-06 1.33830225e-04
 4.43184839e-03 5.39909662e-02 2.41970723e-01 3.98942278e-01
 2.41970723e-01 5.39909662e-02 4.43184839e-03 1.33830225e-04
 1.48671951e-06 6.07588282e-09 9.13472036e-12]
